In [47]:
import pandas as pd
import plotly

In [54]:
df = pd.read_excel('data/df_sank_red.xlsx')
df.columns = df.columns.str.lower()
df.head()

,comb,count,nr_inds
0,UTI_UTI,167301,2
1,MISSING_MISSING,138878,2
2,RTI_RTI,41296,2
3,UTI_MISSING,37213,2
4,MISSING_UTI,31285,2


In [16]:
df = pd.read_csv('data/example_data.csv')


In [55]:
df = df.assign(lvl1 = '', lvl2 = '', lvl3 = '', lvl4 = '', lvl5 = '')
df.head()

,comb,count,nr_inds,lvl1,lvl2,lvl3,lvl4,lvl5
0,UTI_UTI,167301,2,,,,,
1,MISSING_MISSING,138878,2,,,,,
2,RTI_RTI,41296,2,,,,,
3,UTI_MISSING,37213,2,,,,,
4,MISSING_UTI,31285,2,,,,,


In [60]:
for tup in df.itertuples():
    i = tup.Index
    string_to_split = tup.comb
    list_of_v = string_to_split.split('_')
    for j in range(len(list_of_v)):
        if j > 4:
            break
        df.loc[i, 'lvl'+str(j+1)] = str(list_of_v[j])+ '_' + str(j+1)
            
        
        

In [61]:
df.head()

,comb,count,nr_inds,lvl1,lvl2,lvl3,lvl4,lvl5
0,UTI_UTI,167301,2,UTI_1,UTI_2,,,
1,MISSING_MISSING,138878,2,MISSING_1,MISSING_2,,,
2,RTI_RTI,41296,2,RTI_1,RTI_2,,,
3,UTI_MISSING,37213,2,UTI_1,MISSING_2,,,
4,MISSING_UTI,31285,2,MISSING_1,UTI_2,,,


In [38]:
def genSankey(df,cat_cols=[],value_cols='',title='Sankey Diagram'):
    # maximum of 6 value cols -> 6 colors
    colorPalette = ['#4B8BBE','#306998','#FFE873','#FFD43B','#646464']
    labelList = []
    colorNumList = []
    for catCol in cat_cols:
        labelListTemp =  list(set(df[catCol].values))
        colorNumList.append(len(labelListTemp))
        labelList = labelList + labelListTemp
        
    # remove duplicates from labelList
    labelList = list(dict.fromkeys(labelList))
    
    # define colors based on number of levels
    colorList = []
    for idx, colorNum in enumerate(colorNumList):
        colorList = colorList + [colorPalette[idx]]*colorNum
        
    # transform df into a source-target pair
    for i in range(len(cat_cols)-1):
        if i==0:
            sourceTargetDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            sourceTargetDf.columns = ['source','target','count']
        else:
            tempDf = df[[cat_cols[i],cat_cols[i+1],value_cols]]
            tempDf.columns = ['source','target','count']
            sourceTargetDf = pd.concat([sourceTargetDf,tempDf])
        sourceTargetDf = sourceTargetDf.groupby(['source','target']).agg({'count':'sum'}).reset_index()
        
    # add index for source-target pair
    sourceTargetDf['sourceID'] = sourceTargetDf['source'].apply(lambda x: labelList.index(x))
    sourceTargetDf['targetID'] = sourceTargetDf['target'].apply(lambda x: labelList.index(x))
    
    # creating the sankey diagram
    data = dict(
        type='sankey',
        node = dict(
          pad = 15,
          thickness = 20,
          line = dict(
            color = "black",
            width = 0.5
          ),
          label = labelList,
          color = colorList
        ),
        link = dict(
          source = sourceTargetDf['sourceID'],
          target = sourceTargetDf['targetID'],
          value = sourceTargetDf['count']
        )
      )
    
    layout =  dict(
        title = title,
        font = dict(
          size = 10
        )
    )
       
    fig = dict(data=[data], layout=layout)
    return fig

In [62]:
fig = genSankey(df, cat_cols=['lvl1','lvl2', 'lvl3', 'lvl4', 'lvl5'],value_cols='count',title='Sankey Diagram')

In [63]:
plotly.offline.plot(fig, validate=False)

'temp-plot.html'